# Data from Quandl

In [32]:
import quandl
import pandas as pd
from itertools import product
from pprint import pprint
from uszipcode import ZipcodeSearchEngine
import csv

In [2]:
def get_quandl_code(area_category, area_code, indicator_code):
    return f"ZILLOW/{area_category}{area_code}_{indicator_code}"

In [3]:
# init. an instance of ZipcodeSearchEngine
search = ZipcodeSearchEngine()

# define quandl api key
API_KEY = "XL_c6FZRjuz6WYRTJB_a"

# set cities to query
# cities_states = [dict(state="Texas", city="Austin"),
#                  dict(state="Texas", city="Houston"),
#                  dict(state="Texas", city="Dallas")]
cities_states = [dict(state="Texas", city="Austin")]

zips = []

# get zip codes per query city
df = pd.DataFrame()
index = 0
for city_state in cities_states:
    zip_codes = search.by_city_and_state(state=city_state["state"],
                                         city=city_state["city"],
                                         returns=200)
    
    for zip_code in zip_codes:
        df = df.append(pd.DataFrame(zip_code.to_dict(), index=[index]))
        index += 1
#         print(zip_code["Zipcode"])
        # Storing all the zip_codes generated
        zips.append(zip_code["Zipcode"])


area_category = "Z"
area_code = zip_code

## Get Latitude and Longitude per zip code

In [94]:
lati = []
long = []
for z in zips:
    search = ZipcodeSearchEngine()
    zipcode = search.by_zipcode(z)
#     print(zipcode)
    lati.append(zipcode["Latitude"])
    long.append(zipcode["Longitude"])

In [95]:
# Printing the latitude and longitude for each zipcode
zips_df = pd.DataFrame(zips, columns=["Zip Code"])
lati_df = pd.DataFrame(lati, columns=["Latitude"])
long_df = pd.DataFrame(long, columns=["Longitude"])
lat_long = pd.concat([zips_df, lati_df, long_df], axis=1)
lat_long

,Zip Code,Latitude,Longitude
0,78701,30.272921,-97.744386
1,78702,30.260353,-97.714515
2,78703,30.291533,-97.768836
3,78704,30.245673,-97.768836
4,78705,30.296171,-97.738954
5,78712,30.283515,-97.734881
6,78717,30.491658,-97.774270
7,78719,30.159025,-97.681943
8,78721,30.273737,-97.681943
9,78722,30.292043,-97.711800


In [93]:
# Export the "lat and lng" values to a csv file
lat_long.to_csv("../data/int/0_Lat_Long.csv", index = False ,header = True)

## Zillow Home Value Index - All Homes|ZHVIAH
The Zillow Home Value Index is the median Zestimate valuation for a given geographic area on a given day. <https://www.zillow.com/home-values/>

For now the code is only working for the home value index (Zillow Home Value Index - All Homes). Change: indicator_code to check out some other values.

In [4]:
# For all the zip codes obtained
x=0
data_zip = []
data_data = []
for z in zips:
    # set quandl api key
    quandl.ApiConfig.api_key = API_KEY
    # set the indicator code to query
    # indicator: Zillow Home Value Index
    indicator_code = "ZHVIAH"
    # get quandl code to query
#     quandl_code = get_quandl_code(area_category, zips[x], indicator_code)
    quandl_code = get_quandl_code(area_category, z, indicator_code)
#     print(z)
    # when it finds a zip code where there is not houses for sale (example: 78712 UT)
    try:
        # get housing indicator
        data = quandl.get(quandl_code, start_date="1800-01-01", end_date="2018-04-30")
        pprint(data)
        data_zip.append(z)
        # Appending all the data for every zip code
        data_add_zip = data
        data_add_zip["Zip Code"] = z
        data_data.append(data_add_zip)
    except (KeyboardInterrupt, SystemExit):
        raise
    except Exception as exp:
        print(f"caught exception {exp}")
        continue
    x +=1

# Copying the data stored 
home_value_concat = data_data

               Value
Date                
1996-04-30  152500.0
1996-05-31  153600.0
1996-06-30  154600.0
1996-07-31  155500.0
1996-08-31  156300.0
1996-09-30  157300.0
1996-10-31  158300.0
1996-11-30  159300.0
1996-12-31  160400.0
1997-01-31  161900.0
1997-02-28  163500.0
1997-03-31  164800.0
1997-04-30  166300.0
1997-05-31  167700.0
1997-06-30  169000.0
1997-07-31  170200.0
1997-08-31  171500.0
1997-09-30  172900.0
1997-10-31  174500.0
1997-11-30  176100.0
1997-12-31  178000.0
1998-01-31  180200.0
1998-02-28  182700.0
1998-03-31  185000.0
1998-04-30  187500.0
1998-05-31  189800.0
1998-06-30  191800.0
1998-07-31  193700.0
1998-08-31  195500.0
1998-09-30  197300.0
...              ...
2015-11-30  367200.0
2015-12-31  370300.0
2016-01-31  372800.0
2016-02-29  375900.0
2016-03-31  375300.0
2016-04-30  369600.0
2016-05-31  366400.0
2016-06-30  370000.0
2016-07-31  373400.0
2016-08-31  373300.0
2016-09-30  370000.0
2016-10-31  366900.0
2016-11-30  365800.0
2016-12-31  366600.0
2017-01-31  3

               Value
Date                
1996-04-30   66700.0
1996-05-31   66500.0
1996-06-30   66300.0
1996-07-31   66200.0
1996-08-31   66200.0
1996-09-30   66200.0
1996-10-31   66200.0
1996-11-30   66400.0
1996-12-31   66700.0
1997-01-31   67100.0
1997-02-28   67700.0
1997-03-31   68400.0
1997-04-30   69200.0
1997-05-31   70100.0
1997-06-30   71100.0
1997-07-31   72200.0
1997-08-31   73400.0
1997-09-30   74600.0
1997-10-31   75800.0
1997-11-30   77100.0
1997-12-31   78400.0
1998-01-31   79700.0
1998-02-28   81000.0
1998-03-31   82300.0
1998-04-30   83500.0
1998-05-31   84600.0
1998-06-30   85600.0
1998-07-31   86600.0
1998-08-31   87400.0
1998-09-30   88100.0
...              ...
2015-11-30  212700.0
2015-12-31  213800.0
2016-01-31  216800.0
2016-02-29  222100.0
2016-03-31  228200.0
2016-04-30  232400.0
2016-05-31  237300.0
2016-06-30  244300.0
2016-07-31  250300.0
2016-08-31  254000.0
2016-09-30  257200.0
2016-10-31  259900.0
2016-11-30  261700.0
2016-12-31  264500.0
2017-01-31  2

               Value
Date                
1996-04-30  136600.0
1996-05-31  136500.0
1996-06-30  136400.0
1996-07-31  136300.0
1996-08-31  136100.0
1996-09-30  136000.0
1996-10-31  135900.0
1996-11-30  135900.0
1996-12-31  135900.0
1997-01-31  135900.0
1997-02-28  136000.0
1997-03-31  136100.0
1997-04-30  136200.0
1997-05-31  136400.0
1997-06-30  136600.0
1997-07-31  136900.0
1997-08-31  137200.0
1997-09-30  137500.0
1997-10-31  137800.0
1997-11-30  138100.0
1997-12-31  138300.0
1998-01-31  138500.0
1998-02-28  138600.0
1998-03-31  138600.0
1998-04-30  138600.0
1998-05-31  138600.0
1998-06-30  138500.0
1998-07-31  138400.0
1998-08-31  138400.0
1998-09-30  138400.0
...              ...
2015-11-30  220300.0
2015-12-31  220300.0
2016-01-31  221200.0
2016-02-29  223400.0
2016-03-31  226700.0
2016-04-30  230000.0
2016-05-31  233200.0
2016-06-30  236600.0
2016-07-31  239500.0
2016-08-31  242100.0
2016-09-30  243800.0
2016-10-31  244700.0
2016-11-30  246300.0
2016-12-31  249400.0
2017-01-31  2

               Value
Date                
1996-04-30  753500.0
1996-05-31  740800.0
1996-06-30  728700.0
1996-07-31  717100.0
1996-08-31  706500.0
1996-09-30  697200.0
1996-10-31  689100.0
1996-11-30  683400.0
1996-12-31  678100.0
1997-01-31  673600.0
1997-02-28  672000.0
1997-03-31  673800.0
1997-04-30  676700.0
1997-05-31  680400.0
1997-06-30  684500.0
1997-07-31  688300.0
1997-08-31  691300.0
1997-09-30  692600.0
1997-10-31  691800.0
1997-11-30  689200.0
1997-12-31  682700.0
1998-01-31  672900.0
1998-02-28  662000.0
1998-03-31  650700.0
1998-04-30  636900.0
1998-05-31  621700.0
1998-06-30  604900.0
1998-07-31  586900.0
1998-08-31  568100.0
1998-09-30  549000.0
...              ...
2015-11-30  431000.0
2015-12-31  434200.0
2016-01-31  438600.0
2016-02-29  442300.0
2016-03-31  445600.0
2016-04-30  449400.0
2016-05-31  453300.0
2016-06-30  456700.0
2016-07-31  459600.0
2016-08-31  462200.0
2016-09-30  463700.0
2016-10-31  464200.0
2016-11-30  463300.0
2016-12-31  461000.0
2017-01-31  4

               Value
Date                
1996-04-30   85900.0
1996-05-31   85800.0
1996-06-30   85700.0
1996-07-31   85600.0
1996-08-31   85500.0
1996-09-30   85300.0
1996-10-31   85100.0
1996-11-30   84900.0
1996-12-31   84800.0
1997-01-31   84600.0
1997-02-28   84400.0
1997-03-31   84200.0
1997-04-30   84000.0
1997-05-31   83900.0
1997-06-30   83900.0
1997-07-31   83900.0
1997-08-31   84000.0
1997-09-30   84200.0
1997-10-31   84500.0
1997-11-30   84700.0
1997-12-31   85100.0
1998-01-31   85400.0
1998-02-28   85600.0
1998-03-31   85800.0
1998-04-30   85900.0
1998-05-31   86000.0
1998-06-30   85900.0
1998-07-31   85700.0
1998-08-31   85400.0
1998-09-30   85000.0
...              ...
2015-11-30  145400.0
2015-12-31  147000.0
2016-01-31  148900.0
2016-02-29  150400.0
2016-03-31  151100.0
2016-04-30  152100.0
2016-05-31  154200.0
2016-06-30  157800.0
2016-07-31  161300.0
2016-08-31  162800.0
2016-09-30  162100.0
2016-10-31  162100.0
2016-11-30  162900.0
2016-12-31  162900.0
2017-01-31  1

               Value
Date                
1996-04-30  106600.0
1996-05-31  107000.0
1996-06-30  107400.0
1996-07-31  107900.0
1996-08-31  108300.0
1996-09-30  108600.0
1996-10-31  108900.0
1996-11-30  109200.0
1996-12-31  109500.0
1997-01-31  110000.0
1997-02-28  110300.0
1997-03-31  110600.0
1997-04-30  111100.0
1997-05-31  111800.0
1997-06-30  112600.0
1997-07-31  113600.0
1997-08-31  114800.0
1997-09-30  116100.0
1997-10-31  117700.0
1997-11-30  119500.0
1997-12-31  121400.0
1998-01-31  123600.0
1998-02-28  125800.0
1998-03-31  127900.0
1998-04-30  130000.0
1998-05-31  132000.0
1998-06-30  134000.0
1998-07-31  135900.0
1998-08-31  137700.0
1998-09-30  139400.0
...              ...
2015-11-30  378200.0
2015-12-31  380700.0
2016-01-31  382500.0
2016-02-29  385000.0
2016-03-31  389500.0
2016-04-30  394600.0
2016-05-31  398400.0
2016-06-30  401200.0
2016-07-31  401800.0
2016-08-31  400200.0
2016-09-30  398900.0
2016-10-31  399200.0
2016-11-30  399000.0
2016-12-31  399000.0
2017-01-31  4

               Value
Date                
1996-04-30  245500.0
1996-05-31  249200.0
1996-06-30  253300.0
1996-07-31  257500.0
1996-08-31  261300.0
1996-09-30  264600.0
1996-10-31  267100.0
1996-11-30  268800.0
1996-12-31  269800.0
1997-01-31  270200.0
1997-02-28  269800.0
1997-03-31  268700.0
1997-04-30  267300.0
1997-05-31  265600.0
1997-06-30  264000.0
1997-07-31  262600.0
1997-08-31  261500.0
1997-09-30  260800.0
1997-10-31  260500.0
1997-11-30  260400.0
1997-12-31  260600.0
1998-01-31  260800.0
1998-02-28  260500.0
1998-03-31  259700.0
1998-04-30  258200.0
1998-05-31  255800.0
1998-06-30  252500.0
1998-07-31  248300.0
1998-08-31  243500.0
1998-09-30  238400.0
...              ...
2015-11-30  381700.0
2015-12-31  383600.0
2016-01-31  385400.0
2016-02-29  387400.0
2016-03-31  390200.0
2016-04-30  393900.0
2016-05-31  398200.0
2016-06-30  402100.0
2016-07-31  405300.0
2016-08-31  408600.0
2016-09-30  411200.0
2016-10-31  413200.0
2016-11-30  414500.0
2016-12-31  414700.0
2017-01-31  4

In [5]:
# Concatenating all the data
#data_concat = data_data
home_value_concat = pd.concat(home_value_concat)
# Getting month and year from the column "date"
home_value_concat['Year'] = pd.DatetimeIndex(home_value_concat.index).year
home_value_concat['Month'] = pd.DatetimeIndex(home_value_concat.index).month
rename_home_value_concat = home_value_concat.rename(columns={"Value":"Home Value Index"})
rename_home_value_concat

,Home Value Index,Zip Code,Year,Month
Date,,,,
1996-04-30,152500.0,78701,1996,4
1996-05-31,153600.0,78701,1996,5
1996-06-30,154600.0,78701,1996,6
1996-07-31,155500.0,78701,1996,7
1996-08-31,156300.0,78701,1996,8
1996-09-30,157300.0,78701,1996,9
1996-10-31,158300.0,78701,1996,10
1996-11-30,159300.0,78701,1996,11
1996-12-31,160400.0,78701,1996,12


In [6]:
# Export the "home value" data to a csv file
rename_home_value_concat.to_csv("../data/int/output_quandl/1_Output.home_value.csv", index = False ,header = True)

## Inventory Measure (Public)|IMP

In [7]:
# For all the zip codes obtained
x=0
data_zip = []
data_data = []
for z in zips:
    # set quandl api key
    quandl.ApiConfig.api_key = API_KEY
    # set the indicator code to query
    # indicator: Zillow Home Value Index
    indicator_code = "IMP"
    # get quandl code to query
#     quandl_code = get_quandl_code(area_category, zips[x], indicator_code)
    quandl_code = get_quandl_code(area_category, z, indicator_code)
#     print(z)
    # when it finds a zip code where there is not houses for sale (example: 78712 UT)
    try:
        # get housing indicator
        data = quandl.get(quandl_code, start_date="1800-01-01", end_date="2018-04-30")
        pprint(data)
        data_zip.append(z)
        # Appending all the data for every zip code
        data_add_zip = data
        data_add_zip["Zip Code"] = z
        data_data.append(data_add_zip)
    except (KeyboardInterrupt, SystemExit):
        raise
    except Exception as exp:
        print(f"caught exception {exp}")
        continue
    x +=1

# Copying the data stored 
inventory_concat = data_data

caught exception (Status 404) (Quandl Error QECx02) You have submitted an incorrect Quandl code. Please check your Quandl codes and try again.
            Value
Date             
2010-01-31  161.0
2010-02-28  158.0
2010-03-31  173.0
2010-04-30  178.0
2010-05-31  184.0
2010-06-30  177.0
2010-07-31  166.0
2010-08-31  145.0
2010-09-30  140.0
2010-10-31  142.0
2010-11-30  136.0
2010-12-31  130.0
2011-01-31  132.0
2011-02-28  130.0
2011-03-31  131.0
2011-04-30  132.0
2011-05-31  135.0
2011-06-30  124.0
2011-07-31  124.0
2011-08-31  118.0
2011-09-30  111.0
2011-10-31  101.0
2011-11-30   95.0
2011-12-31   84.0
2012-01-31   68.0
2012-02-29   62.0
2012-03-31   68.0
2012-04-30   57.0
2012-05-31   56.0
2012-06-30   53.0
...           ...
2015-11-30  112.0
2015-12-31  101.0
2016-01-31   88.0
2016-02-29   94.0
2016-03-31   85.0
2016-04-30   88.0
2016-05-31   78.0
2016-06-30   96.0
2016-07-31  115.0
2016-08-31  152.0
2016-09-30  157.0
2016-10-31  164.0
2016-11-30  161.0
2016-12-31  159.0
2017-01-31 

            Value
Date             
2010-01-31  150.0
2010-02-28  155.0
2010-03-31  173.0
2010-04-30  196.0
2010-05-31  210.0
2010-06-30  204.0
2010-07-31  220.0
2010-08-31  213.0
2010-09-30  213.0
2010-10-31  207.0
2010-11-30  194.0
2010-12-31  176.0
2011-01-31  164.0
2011-02-28  162.0
2011-03-31  183.0
2011-04-30  196.0
2011-05-31  208.0
2011-06-30  217.0
2011-07-31  216.0
2011-08-31  208.0
2011-09-30  178.0
2011-10-31  173.0
2011-11-30  152.0
2011-12-31  138.0
2012-01-31  136.0
2012-02-29  138.0
2012-03-31  141.0
2012-04-30  117.0
2012-05-31  119.0
2012-06-30  114.0
...           ...
2015-11-30   61.0
2015-12-31   56.0
2016-01-31   48.0
2016-02-29   54.0
2016-03-31   58.0
2016-04-30   76.0
2016-05-31   74.0
2016-06-30   98.0
2016-07-31  102.0
2016-08-31  102.0
2016-09-30  108.0
2016-10-31  112.0
2016-11-30   93.0
2016-12-31   76.0
2017-01-31   78.0
2017-02-28   79.0
2017-03-31   86.0
2017-04-30   92.0
2017-05-31  106.0
2017-06-30  114.0
2017-07-31  105.0
2017-08-31  105.0
2017-09-30

caught exception (Status 404) (Quandl Error QECx02) You have submitted an incorrect Quandl code. Please check your Quandl codes and try again.
caught exception (Status 404) (Quandl Error QECx02) You have submitted an incorrect Quandl code. Please check your Quandl codes and try again.
caught exception (Status 404) (Quandl Error QECx02) You have submitted an incorrect Quandl code. Please check your Quandl codes and try again.
            Value
Date             
2010-01-31  184.0
2010-02-28  182.0
2010-03-31  205.0
2010-04-30  233.0
2010-05-31  234.0
2010-06-30  250.0
2010-07-31  254.0
2010-08-31  230.0
2010-09-30  229.0
2010-10-31  242.0
2010-11-30  256.0
2010-12-31  242.0
2011-01-31  234.0
2011-02-28  256.0
2011-03-31  234.0
2011-04-30  250.0
2011-05-31  240.0
2011-06-30  262.0
2011-07-31  259.0
2011-08-31  217.0
2011-09-30  208.0
2011-10-31  201.0
2011-11-30  172.0
2011-12-31  163.0
2012-01-31  146.0
2012-02-29  138.0
2012-03-31  130.0
2012-04-30  114.0
2012-05-31  112.0
2012-06-30  1

caught exception (Status 404) (Quandl Error QECx02) You have submitted an incorrect Quandl code. Please check your Quandl codes and try again.
            Value
Date             
2010-01-31  125.0
2010-02-28  120.0
2010-03-31  132.0
2010-04-30  134.0
2010-05-31  120.0
2010-06-30  126.0
2010-07-31  130.0
2010-08-31  127.0
2010-09-30  133.0
2010-10-31  126.0
2010-11-30  119.0
2010-12-31  106.0
2011-01-31  104.0
2011-02-28  110.0
2011-03-31  106.0
2011-04-30  110.0
2011-05-31  112.0
2011-06-30  112.0
2011-07-31  116.0
2011-08-31  115.0
2011-09-30   98.0
2011-10-31   88.0
2011-11-30   86.0
2011-12-31   72.0
2012-01-31   60.0
2012-02-29   63.0
2012-03-31   64.0
2012-04-30   62.0
2012-05-31   71.0
2012-06-30   64.0
...           ...
2015-11-30   41.0
2015-12-31   39.0
2016-01-31   42.0
2016-02-29   32.0
2016-03-31   35.0
2016-04-30   44.0
2016-05-31   36.0
2016-06-30   33.0
2016-07-31   42.0
2016-08-31   49.0
2016-09-30   56.0
2016-10-31   52.0
2016-11-30   42.0
2016-12-31   36.0
2017-01-31 

In [8]:
# Concatenating all the data
#inventory_concat = data_data
inventory_concat = pd.concat(inventory_concat)
# Getting month and year from the column "date"
inventory_concat['Year'] = pd.DatetimeIndex(inventory_concat.index).year
inventory_concat['Month'] = pd.DatetimeIndex(inventory_concat.index).month
rename_inventory_concat = inventory_concat.rename(columns={"Value":"Inventory Measure"})
rename_inventory_concat

,Inventory Measure,Zip Code,Year,Month
Date,,,,
2010-01-31,161.0,78702,2010,1
2010-02-28,158.0,78702,2010,2
2010-03-31,173.0,78702,2010,3
2010-04-30,178.0,78702,2010,4
2010-05-31,184.0,78702,2010,5
2010-06-30,177.0,78702,2010,6
2010-07-31,166.0,78702,2010,7
2010-08-31,145.0,78702,2010,8
2010-09-30,140.0,78702,2010,9


In [9]:
# Export the "home value" data to a csv file
rename_inventory_concat.to_csv("../data/int/output_quandl/2_Output.inventory_measure.csv", index = False ,header = True)

## Median Price Cut Dollar - All Homes|MPCDAH

In [10]:
# For all the zip codes obtained
x=0
data_zip = []
data_data = []
for z in zips:
    # set quandl api key
    quandl.ApiConfig.api_key = API_KEY
    # set the indicator code to query
    # indicator: Zillow Home Value Index
    indicator_code = "MPCDAH"
    # get quandl code to query
#     quandl_code = get_quandl_code(area_category, zips[x], indicator_code)
    quandl_code = get_quandl_code(area_category, z, indicator_code)
#     print(z)
    # when it finds a zip code where there is not houses for sale (example: 78712 UT)
    try:
        # get housing indicator
        data = quandl.get(quandl_code, start_date="1800-01-01", end_date="2018-04-30")
        pprint(data)
        data_zip.append(z)
        # Appending all the data for every zip code
        data_add_zip = data
        data_add_zip["Zip Code"] = z
        data_data.append(data_add_zip)
    except (KeyboardInterrupt, SystemExit):
        raise
    except Exception as exp:
        print(f"caught exception {exp}")
        continue
    x +=1
    
# Copying the data stored
price_cut_concat = data_data

               Value
Date                
2010-09-30   10000.0
2010-10-31    9950.0
2010-11-30    6000.0
2010-12-31    8700.0
2011-01-31   10000.0
2011-02-28    7500.0
2011-03-31    8900.0
2011-04-30    7550.0
2011-05-31   15000.0
2011-06-30   10700.0
2011-07-31    4575.0
2011-08-31    7402.5
2011-09-30   10000.0
2011-10-31   10000.0
2011-11-30   15100.0
2011-12-31   12505.0
2012-01-31   10000.0
2012-02-29   20000.0
2012-03-31   10000.0
2012-04-30   10000.0
2012-05-31   10000.0
2012-06-30    7050.0
2012-07-31    6500.0
2012-08-31    5000.0
2012-09-30   10000.0
2012-10-31  100000.0
2012-11-30   10750.0
2012-12-31    7600.0
2013-01-31   17600.0
2013-02-28   69250.0
...              ...
2015-03-31   18500.0
2015-04-30   20000.0
2015-05-31   10000.0
2015-06-30   13299.5
2015-07-31   10000.0
2015-08-31   10000.0
2015-09-30   14750.0
2015-10-31   20000.0
2015-11-30   10000.0
2015-12-31   15400.0
2016-01-31   15000.0
2016-02-29   14999.0
2016-03-31   19000.0
2016-04-30   11250.0
2016-05-31   

caught exception (Status 404) (Quandl Error QECx02) You have submitted an incorrect Quandl code. Please check your Quandl codes and try again.
caught exception (Status 404) (Quandl Error QECx02) You have submitted an incorrect Quandl code. Please check your Quandl codes and try again.
              Value
Date               
2010-09-30  19000.0
2010-10-31  16950.0
2010-11-30   6700.0
2010-12-31   5500.0
2011-01-31   9000.0
2011-02-28   5090.0
2011-03-31   7200.0
2011-04-30  12550.0
2011-05-31  10000.0
2011-06-30  10000.0
2011-07-31  10000.0
2011-08-31  19000.0
2011-09-30  10000.0
2011-10-31   7525.0
2011-11-30   4200.0
2011-12-31  12000.0
2012-01-31   6737.5
2012-02-29   7375.0
2012-03-31  21000.0
2012-04-30  10000.0
2012-05-31   7000.0
2012-06-30   5226.0
2012-07-31  10099.0
2012-08-31  10000.0
2012-09-30   9450.0
2012-10-31  10000.0
2012-11-30  10000.0
2012-12-31  10500.0
2013-01-31  10000.0
2013-02-28  17500.0
...             ...
2015-11-30   8250.0
2015-12-31  10500.0
2016-01-31  10

              Value
Date               
2010-09-30  18500.0
2010-10-31  20000.0
2010-11-30  12292.0
2010-12-31  27500.0
2011-01-31  14550.0
2011-02-28  24900.0
2011-03-31  16000.0
2011-04-30  10000.0
2011-05-31  15000.0
2011-06-30  18000.0
2011-07-31  15100.0
2011-08-31  15000.0
2011-09-30  15000.0
2011-10-31  18000.0
2011-11-30   7500.0
2011-12-31  15000.0
2012-01-31  13500.0
2012-02-29  10000.0
2012-03-31  11600.0
2012-04-30  10600.0
2012-05-31  14000.0
2012-06-30  10000.0
2012-07-31  15000.0
2012-08-31  15000.0
2012-09-30  15000.0
2012-10-31  16000.0
2012-11-30  11053.0
2012-12-31   5000.0
2013-01-31  10000.0
2013-02-28  10000.0
...             ...
2015-11-30  15100.0
2015-12-31   5000.0
2016-01-31   9100.0
2016-02-29   9000.0
2016-03-31  15000.0
2016-04-30  15250.0
2016-05-31  15000.0
2016-06-30  14900.0
2016-07-31  10000.0
2016-08-31  10999.0
2016-09-30  10000.0
2016-10-31  10049.5
2016-11-30  11705.0
2016-12-31  15000.0
2017-01-31  10064.0
2017-02-28  11850.0
2017-03-31  11000.0


              Value
Date               
2010-09-30  10000.0
2010-10-31  10000.0
2010-11-30  10000.0
2010-12-31   5000.0
2011-01-31  12500.0
2011-02-28  11950.0
2011-03-31   8500.0
2011-04-30   5000.0
2011-05-31   8250.0
2011-06-30   9100.0
2011-07-31   5750.0
2011-08-31   9300.0
2011-09-30  10000.0
2011-10-31   5900.0
2011-11-30   8875.0
2011-12-31  10000.0
2012-01-31   6400.0
2012-02-29   4000.0
2012-03-31   6750.0
2012-04-30   9550.0
2012-05-31   5000.0
2012-06-30   6500.0
2012-07-31  10000.0
2012-08-31   5550.0
2012-09-30   8425.0
2012-10-31   6000.0
2012-11-30   5097.5
2012-12-31   3950.5
2013-01-31   5000.0
2013-02-28   7000.0
...             ...
2015-11-30   8000.0
2015-12-31  10000.0
2016-01-31   8500.0
2016-02-29  15600.0
2016-03-31  10000.0
2016-04-30  10950.0
2016-05-31  10000.0
2016-06-30  10000.0
2016-07-31  11000.0
2016-08-31  10000.0
2016-09-30  10000.0
2016-10-31  12500.0
2016-11-30  10000.0
2016-12-31  12500.0
2017-01-31  10000.0
2017-02-28  10500.0
2017-03-31  10000.0


In [11]:
# Concatenating all the data
#data_price_cut_concat = data_data
price_cut_concat = pd.concat(price_cut_concat)
# Getting month and year from the column "date"
price_cut_concat['Year'] = pd.DatetimeIndex(price_cut_concat.index).year
price_cut_concat['Month'] = pd.DatetimeIndex(price_cut_concat.index).month
rename_price_cut_concat = price_cut_concat.rename(columns={"Value":"Median Price Cut"})
rename_price_cut_concat

,Median Price Cut,Zip Code,Year,Month
Date,,,,
2010-09-30,10000.0,78701,2010,9
2010-10-31,9950.0,78701,2010,10
2010-11-30,6000.0,78701,2010,11
2010-12-31,8700.0,78701,2010,12
2011-01-31,10000.0,78701,2011,1
2011-02-28,7500.0,78701,2011,2
2011-03-31,8900.0,78701,2011,3
2011-04-30,7550.0,78701,2011,4
2011-05-31,15000.0,78701,2011,5


In [12]:
# Export the "home value" data to a csv file
rename_price_cut_concat.to_csv("../data/int/output_quandl/3_Output.median_price_cut.csv", index = False ,header = True)

## Median Price Of Reduction - All Homes|MPPRAH

In [13]:
# For all the zip codes obtained
x=0
data_zip = []
data_data = []
for z in zips:
    # set quandl api key
    quandl.ApiConfig.api_key = API_KEY
    # set the indicator code to query
    # indicator: Zillow Home Value Index
    indicator_code = "MPPRAH"
    # get quandl code to query
#     quandl_code = get_quandl_code(area_category, zips[x], indicator_code)
    quandl_code = get_quandl_code(area_category, z, indicator_code)
#     print(z)
    # when it finds a zip code where there is not houses for sale (example: 78712 UT)
    try:
        # get housing indicator
        data = quandl.get(quandl_code, start_date="1800-01-01", end_date="2018-04-30")
        pprint(data)
        data_zip.append(z)
        # Appending all the data for every zip code
        data_add_zip = data
        data_add_zip["Zip Code"] = z
        data_data.append(data_add_zip)
    except (KeyboardInterrupt, SystemExit):
        raise
    except Exception as exp:
        print(f"caught exception {exp}")
        continue
    x +=1
    
# Copying the data stored    
price_reduction_concat = data_data

               Value
Date                
2010-09-30  3.572705
2010-10-31  2.864370
2010-11-30  2.521008
2010-12-31  4.299446
2011-01-31  3.386514
2011-02-28  2.482331
2011-03-31  4.549591
2011-04-30  2.406481
2011-05-31  4.799399
2011-06-30  3.584229
2011-07-31  1.668046
2011-08-31  2.470238
2011-09-30  3.177258
2011-10-31  2.570694
2011-11-30  4.624277
2011-12-31  2.923402
2012-01-31  2.942042
2012-02-29  4.264578
2012-03-31  1.395797
2012-04-30  1.803607
2012-05-31  2.962963
2012-06-30  1.691671
2012-07-31  2.023706
2012-08-31  1.724733
2012-09-30  2.669633
2012-10-31  4.008016
2012-11-30  2.710790
2012-12-31  2.788991
2013-01-31  2.214686
2013-02-28  7.671544
...              ...
2015-03-31  3.423681
2015-04-30  3.047992
2015-05-31  2.063983
2015-06-30  2.698453
2015-07-31  1.820359
2015-08-31  2.199111
2015-09-30  3.366340
2015-10-31  3.125000
2015-11-30  2.352941
2015-12-31  3.333333
2016-01-31  2.195652
2016-02-29  2.370073
2016-03-31  3.660886
2016-04-30  2.387287
2016-05-31  2

caught exception (Status 404) (Quandl Error QECx02) You have submitted an incorrect Quandl code. Please check your Quandl codes and try again.
               Value
Date                
2010-09-30  4.000000
2010-10-31  4.740020
2010-11-30  2.004626
2010-12-31  2.018080
2011-01-31  3.151862
2011-02-28  1.817922
2011-03-31  2.062847
2011-04-30  3.875000
2011-05-31  2.521008
2011-06-30  3.261867
2011-07-31  2.515823
2011-08-31  4.099021
2011-09-30  3.302201
2011-10-31  1.470778
2011-11-30  1.682914
2011-12-31  4.145078
2012-01-31  2.535921
2012-02-29  2.736447
2012-03-31  7.015773
2012-04-30  3.095646
2012-05-31  1.668948
2012-06-30  2.005731
2012-07-31  2.673797
2012-08-31  2.105263
2012-09-30  1.982588
2012-10-31  2.634211
2012-11-30  3.394802
2012-12-31  2.729216
2013-01-31  1.817978
2013-02-28  3.310468
...              ...
2015-11-30  1.918708
2015-12-31  2.335929
2016-01-31  2.632272
2016-02-29  2.586285
2016-03-31  2.725681
2016-04-30  1.887149
2016-05-31  2.325896
2016-06-30  2.353

               Value
Date                
2010-09-30  3.990535
2010-10-31  3.432484
2010-11-30  2.576169
2010-12-31  5.202995
2011-01-31  2.796491
2011-02-28  3.846154
2011-03-31  3.482759
2011-04-30  2.324822
2011-05-31  3.040000
2011-06-30  3.338898
2011-07-31  3.061224
2011-08-31  2.970588
2011-09-30  3.096539
2011-10-31  3.464071
2011-11-30  2.039340
2011-12-31  2.883269
2012-01-31  2.636486
2012-02-29  2.178649
2012-03-31  2.439619
2012-04-30  2.631579
2012-05-31  2.574359
2012-06-30  2.222227
2012-07-31  2.818023
2012-08-31  2.769796
2012-09-30  3.574013
2012-10-31  2.816994
2012-11-30  2.534854
2012-12-31  1.187829
2013-01-31  2.293234
2013-02-28  2.165805
...              ...
2015-11-30  2.326220
2015-12-31  1.206273
2016-01-31  1.823647
2016-02-29  1.796748
2016-03-31  2.439024
2016-04-30  2.521402
2016-05-31  2.500313
2016-06-30  2.189813
2016-07-31  1.672241
2016-08-31  2.157143
2016-09-30  2.002002
2016-10-31  1.902505
2016-11-30  2.109499
2016-12-31  2.034884
2017-01-31  1

               Value
Date                
2010-09-30  3.236246
2010-10-31  3.290323
2010-11-30  2.914286
2010-12-31  2.113402
2011-01-31  4.328859
2011-02-28  3.300021
2011-03-31  2.766970
2011-04-30  2.352941
2011-05-31  3.062510
2011-06-30  2.564760
2011-07-31  2.221176
2011-08-31  3.106239
2011-09-30  3.278689
2011-10-31  2.702951
2011-11-30  2.978323
2011-12-31  3.685665
2012-01-31  2.586207
2012-02-29  1.647991
2012-03-31  1.953643
2012-04-30  2.415462
2012-05-31  2.241575
2012-06-30  2.635046
2012-07-31  2.273244
2012-08-31  1.870214
2012-09-30  3.159558
2012-10-31  2.506266
2012-11-30  2.192722
2012-12-31  2.467162
2013-01-31  2.036877
2013-02-28  2.653659
...              ...
2015-11-30  2.000000
2015-12-31  1.890359
2016-01-31  1.394093
2016-02-29  2.606516
2016-03-31  2.336579
2016-04-30  2.840849
2016-05-31  1.810371
2016-06-30  2.044990
2016-07-31  2.491874
2016-08-31  2.771003
2016-09-30  2.760479
2016-10-31  2.539806
2016-11-30  2.328340
2016-12-31  3.283958
2017-01-31  1

In [14]:
# Concatenating all the data
#price_reduction_concat = data_data
price_reduction_concat = pd.concat(price_reduction_concat)
# Getting month and year from the column "date"
price_reduction_concat['Year'] = pd.DatetimeIndex(price_reduction_concat.index).year
price_reduction_concat['Month'] = pd.DatetimeIndex(price_reduction_concat.index).month
rename_price_reduction_concat = price_reduction_concat.rename(columns={"Value":"Median Price of Reduction"})
rename_price_reduction_concat

,Median Price of Reduction,Zip Code,Year,Month
Date,,,,
2010-09-30,3.572705,78701,2010,9
2010-10-31,2.864370,78701,2010,10
2010-11-30,2.521008,78701,2010,11
2010-12-31,4.299446,78701,2010,12
2011-01-31,3.386514,78701,2011,1
2011-02-28,2.482331,78701,2011,2
2011-03-31,4.549591,78701,2011,3
2011-04-30,2.406481,78701,2011,4
2011-05-31,4.799399,78701,2011,5


In [15]:
# Export the "home value" data to a csv file
rename_price_reduction_concat.to_csv("../data/int/output_quandl/4_Output.price_of_reduction.csv", index = False ,header = True)

## Median Listing Price - All Homes|MLPAH

In [16]:
# For all the zip codes obtained
x=0
data_zip = []
data_data = []
for z in zips:
    # set quandl api key
    quandl.ApiConfig.api_key = API_KEY
    # set the indicator code to query
    # indicator: Zillow Home Value Index
    indicator_code = "MLPAH"
    # get quandl code to query
#     quandl_code = get_quandl_code(area_category, zips[x], indicator_code)
    quandl_code = get_quandl_code(area_category, z, indicator_code)
#     print(z)
    # when it finds a zip code where there is not houses for sale (example: 78712 UT)
    try:
        # get housing indicator
        data = quandl.get(quandl_code, start_date="1800-01-01", end_date="2018-04-30")
        pprint(data)
        data_zip.append(z)
        # Appending all the data for every zip code
        data_add_zip = data
        data_add_zip["Zip Code"] = z
        data_data.append(data_add_zip)
    except (KeyboardInterrupt, SystemExit):
        raise
    except Exception as exp:
        print(f"caught exception {exp}")
        continue
    x +=1

# Copying the data stored 
listing_concat = data_data

               Value
Date                
2015-02-28  498000.0
2015-03-31  465000.0
2015-04-30  475000.0
2015-05-31  474750.0
2015-06-30  509999.0
2015-07-31  522247.5
2015-08-31  533700.0
2015-09-30  539900.0
2015-10-31  533280.0
2015-11-30  491950.0
2015-12-31  491950.0
2016-01-31  494900.0
2016-02-29  519950.0
2016-03-31  509950.0
2016-04-30  499500.0
2016-05-31  537000.0
2016-06-30  545000.0
2016-07-31  535000.0
2016-08-31  585000.0
2016-09-30  585000.0
2016-10-31  597450.0
2016-11-30  573450.0
2016-12-31  573450.0
2017-01-31  574950.0
2017-02-28  564500.0
2017-03-31  561950.0
2017-04-30  587950.0
2017-05-31  596850.0
2017-06-30  598000.0
2017-07-31  598700.0
2017-08-31  605000.0
               Value
Date                
2010-01-31  210000.0
2010-02-28  199999.0
2010-03-31  200000.0
2010-04-30  207250.0
2010-05-31  209900.0
2010-06-30  209900.0
2010-07-31  212700.0
2010-08-31  211450.0
2010-09-30  209700.0
2010-10-31  200000.0
2010-11-30  199997.0
2010-12-31  199999.0
2011-01-31  1

               Value
Date                
2016-05-31  227450.0
2016-06-30  219999.5
2016-07-31  219999.5
2016-08-31  236500.0
2016-09-30  244450.0
2016-10-31  239450.0
2016-11-30  213000.0
2016-12-31  229400.0
2017-01-31  217990.0
2017-02-28  221990.0
2017-03-31  220000.0
2017-04-30  232997.5
2017-05-31  247242.5
2017-06-30  243000.0
2017-07-31  234900.0
2017-08-31  239400.0
2017-09-30  247450.0
2017-10-31  220000.0
2017-11-30  219900.0
2017-12-31  222495.0
2018-01-31  221990.0
2018-02-28  222990.0
2018-03-31  224000.0
2018-04-30  224990.0
               Value
Date                
2015-01-31  153500.0
2015-02-28  149500.0
2015-03-31  137000.0
2015-04-30  146500.0
2015-05-31  160000.0
2015-06-30  149999.0
2015-07-31  156900.0
2015-08-31  169900.0
2015-09-30  176492.5
2015-10-31  181490.0
2015-11-30  179245.0
2015-12-31  178245.0
2016-01-31  180990.0
2016-02-29  179495.0
2016-03-31  175000.0
2016-04-30  179120.0
2016-05-31  179512.5
2016-06-30  187510.0
2016-07-31  186344.0
2016-08-31  1

               Value
Date                
2010-01-31  430500.0
2010-02-28  429900.0
2010-03-31  415450.0
2010-04-30  410250.0
2010-05-31  431000.0
2010-06-30  430000.0
2010-07-31  429900.0
2010-08-31  419500.0
2010-09-30  441495.0
2010-10-31  449925.0
2010-11-30  439000.0
2010-12-31  439000.0
2011-01-31  458495.0
2011-02-28  450000.0
2011-03-31  449000.0
2011-04-30  449000.0
2011-05-31  454900.0
2011-06-30  465000.0
2011-07-31  456500.0
2011-08-31  452500.0
2011-09-30  440000.0
2011-10-31  439990.0
2011-11-30  449995.0
2011-12-31  471950.0
2012-01-31  461500.0
2012-02-29  464500.0
2012-03-31  477000.0
2012-04-30  471500.0
2012-05-31  475000.0
2012-06-30  485000.0
...              ...
2015-11-30  534900.0
2015-12-31  514900.0
2016-01-31  525000.0
2016-02-29  539450.0
2016-03-31  550000.0
2016-04-30  565000.0
2016-05-31  550000.0
2016-06-30  565000.0
2016-07-31  575000.0
2016-08-31  599900.0
2016-09-30  605000.0
2016-10-31  587000.0
2016-11-30  597450.0
2016-12-31  597000.0
2017-01-31  5

               Value
Date                
2015-10-31  280000.0
2015-11-30  278990.0
2015-12-31  285000.0
2016-01-31  280990.0
2016-02-29  278990.0
2016-03-31  296990.0
2016-04-30  296990.0
2016-05-31  278812.0
2016-06-30  291995.0
2016-07-31  310950.0
2016-08-31  312000.0
2016-09-30  316495.0
2016-10-31  329900.0
2016-11-30  325000.0
2016-12-31  315000.0
2017-01-31  309900.0
2017-02-28  319945.0
2017-03-31  329450.0
2017-04-30  339900.0
2017-05-31  329900.0
2017-06-30  330000.0
2017-07-31  317498.5
2017-08-31  319000.0
2017-09-30  326020.0
2017-10-31  325045.0
2017-11-30  323500.0
2017-12-31  330000.0
2018-01-31  338295.0
2018-02-28  295990.0
2018-03-31  289450.0
caught exception (Status 404) (Quandl Error QECx02) You have submitted an incorrect Quandl code. Please check your Quandl codes and try again.
               Value
Date                
2013-11-30  152945.0
2013-12-31  156995.0
2014-01-31  149990.0
2014-02-28  149900.0
2014-03-31  144000.0
2014-04-30  150500.0
2014-05-31  16999

               Value
Date                
2016-09-30  455000.0
2016-10-31  442500.0
2016-11-30  439249.5
2016-12-31  439000.0
2017-01-31  414900.0
2017-02-28  465000.0
2017-03-31  484950.0
2017-04-30  489900.0
2017-05-31  489949.5
2017-06-30  477350.0
2017-07-31  450000.0
2017-08-31  424950.0
2017-09-30  409475.0
2017-10-31  399950.0
2017-11-30  419000.0
2017-12-31  372350.0
2018-01-31  399000.0
2018-02-28  399000.0
2018-03-31  400000.0
2018-04-30  399000.0
               Value
Date                
2015-02-28  459445.0
2015-03-31  449000.0
2015-04-30  421450.0
2015-05-31  456950.0
2015-06-30  449500.0
2015-07-31  425000.0
2015-08-31  399990.0
2015-09-30  449000.0
2015-10-31  477450.0
2015-11-30  469000.0
2015-12-31  469000.0
2016-01-31  419000.0
2016-02-29  420000.0
2016-03-31  399499.5
2016-04-30  399999.0
2016-05-31  418700.0
2016-06-30  449000.0
2016-07-31  440000.0
2016-08-31  450000.0
2016-09-30  449950.0
2016-10-31  435000.0
2016-11-30  448500.0
2016-12-31  449500.0
2017-01-31  4

In [17]:
# Concatenating all the data
#data_listing_concat = data_data
listing_concat = pd.concat(listing_concat)
# Getting month and year from the column "date"
listing_concat['Year'] = pd.DatetimeIndex(listing_concat.index).year
listing_concat['Month'] = pd.DatetimeIndex(listing_concat.index).month
rename_listing_concat = listing_concat.rename(columns={"Value":"Median Listing Price"})
rename_listing_concat

,Median Listing Price,Zip Code,Year,Month
Date,,,,
2015-02-28,498000.0,78701,2015,2
2015-03-31,465000.0,78701,2015,3
2015-04-30,475000.0,78701,2015,4
2015-05-31,474750.0,78701,2015,5
2015-06-30,509999.0,78701,2015,6
2015-07-31,522247.5,78701,2015,7
2015-08-31,533700.0,78701,2015,8
2015-09-30,539900.0,78701,2015,9
2015-10-31,533280.0,78701,2015,10


In [18]:
# Export the "home value" data to a csv file
rename_listing_concat.to_csv("../data/int/output_quandl/5_Output.median_listing_price.csv", index = False ,header = True)

## Median Listing Price Per Square Foot - All Homes|MLPFAH

In [19]:
# For all the zip codes obtained
x=0
data_zip = []
data_data = []
for z in zips:
    # set quandl api key
    quandl.ApiConfig.api_key = API_KEY
    # set the indicator code to query
    # indicator: Zillow Home Value Index
    indicator_code = "MLPFAH"
    # get quandl code to query
#     quandl_code = get_quandl_code(area_category, zips[x], indicator_code)
    quandl_code = get_quandl_code(area_category, z, indicator_code)
#     print(z)
    # when it finds a zip code where there is not houses for sale (example: 78712 UT)
    try:
        # get housing indicator
        data = quandl.get(quandl_code, start_date="1800-01-01", end_date="2018-04-30")
        pprint(data)
        data_zip.append(z)
        # Appending all the data for every zip code
        data_add_zip = data
        data_add_zip["Zip Code"] = z
        data_data.append(data_add_zip)
    except (KeyboardInterrupt, SystemExit):
        raise
    except Exception as exp:
        print(f"caught exception {exp}")
        continue
    x +=1

# Copying the data stored
listing_sqf_concat = data_data

                 Value
Date                  
2010-01-31  342.530800
2010-02-28  342.245287
2010-03-31  339.803625
2010-04-30  330.220815
2010-05-31  332.045089
2010-06-30  329.467580
2010-07-31  327.641105
2010-08-31  331.973684
2010-09-30  331.474597
2010-10-31  332.554658
2010-11-30  327.518428
2010-12-31  329.023637
2011-01-31  319.238856
2011-02-28  318.965517
2011-03-31  306.124853
2011-04-30  316.212014
2011-05-31  317.570093
2011-06-30  325.816485
2011-07-31  326.516330
2011-08-31  342.972195
2011-09-30  349.059140
2011-10-31  351.286174
2011-11-30  348.402818
2011-12-31  329.670330
2012-01-31  329.810412
2012-02-29  327.616929
2012-03-31  330.687831
2012-04-30  346.828610
2012-05-31  375.370971
2012-06-30  380.810861
...                ...
2015-03-31  487.089202
2015-04-30  504.189294
2015-05-31  510.646688
2015-06-30  518.123344
2015-07-31  517.685343
2015-08-31  496.826768
2015-09-30  525.962399
2015-10-31  516.428177
2015-11-30  491.300797
2015-12-31  489.162131
2016-01-31 

caught exception (Status 404) (Quandl Error QECx02) You have submitted an incorrect Quandl code. Please check your Quandl codes and try again.
                 Value
Date                  
2014-11-30  186.923077
2014-12-31  195.750648
2015-01-31  192.345338
2015-02-28  197.916667
2015-03-31  199.846681
2015-04-30  209.536495
2015-05-31  210.629329
2015-06-30  208.565072
2015-07-31  215.219907
2015-08-31  215.277778
2015-09-30  214.786303
2015-10-31  215.109954
2015-11-30  214.644652
2015-12-31  215.730822
2016-01-31  225.349206
2016-02-29  226.507786
2016-03-31  227.669577
2016-04-30  236.303403
2016-05-31  242.300000
2016-06-30  246.098300
2016-07-31  247.422680
2016-08-31  248.962524
2016-09-30  245.054046
2016-10-31  240.458015
2016-11-30  251.026777
2016-12-31  246.772152
2017-01-31  248.178089
2017-02-28  252.937371
2017-03-31  254.603322
2017-04-30  253.091509
2017-05-31  265.356602
2017-06-30  263.460187
2017-07-31  265.276725
2017-08-31  267.507645
2017-09-30  270.955970
2017-1

                 Value
Date                  
2010-01-31  102.503016
2010-02-28  103.149272
2010-03-31  102.971846
2010-04-30  103.033074
2010-05-31  103.094303
2010-06-30  102.279073
2010-07-31  101.655152
2010-08-31  101.425121
2010-09-30  101.395960
2010-10-31  101.077186
2010-11-30  100.847817
2010-12-31  101.125938
2011-01-31  101.060158
2011-02-28  100.503022
2011-03-31  100.000000
2011-04-30   98.432346
2011-05-31   98.353156
2011-06-30   99.647355
2011-07-31   99.690177
2011-08-31   99.061390
2011-09-30   97.590631
2011-10-31   97.150259
2011-11-30   97.510373
2011-12-31   97.972973
2012-01-31   96.580930
2012-02-29   96.576239
2012-03-31   96.840655
2012-04-30   99.172185
2012-05-31  100.394564
2012-06-30  101.659529
...                ...
2015-11-30  146.091752
2015-12-31  146.400269
2016-01-31  137.788018
2016-02-29  132.870614
2016-03-31  132.632961
2016-04-30  135.135135
2016-05-31  137.889887
2016-06-30  140.469417
2016-07-31  144.621709
2016-08-31  143.283582
2016-09-30 

                 Value
Date                  
2010-01-31  116.878495
2010-02-28  114.916986
2010-03-31  115.399033
2010-04-30  116.570857
2010-05-31  119.552642
2010-06-30  118.539326
2010-07-31  117.639222
2010-08-31  116.822430
2010-09-30  114.567858
2010-10-31  112.701386
2010-11-30  113.123094
2010-12-31  113.123094
2011-01-31  111.925287
2011-02-28  108.275039
2011-03-31  101.306239
2011-04-30  108.060685
2011-05-31  106.178001
2011-06-30  108.370703
2011-07-31  109.765289
2011-08-31  110.808896
2011-09-30  104.719764
2011-10-31  104.048071
2011-11-30  100.102529
2011-12-31  102.382443
2012-01-31   97.958628
2012-02-29  105.128359
2012-03-31  114.352183
2012-04-30  116.906662
2012-05-31  118.906064
2012-06-30  117.935103
...                ...
2015-11-30  166.481069
2015-12-31  178.147268
2016-01-31  171.776386
2016-02-29  167.039106
2016-03-31  159.589041
2016-04-30  159.589041
2016-05-31  173.896400
2016-06-30  187.040748
2016-07-31  187.040748
2016-08-31  169.082126
2016-09-30 

                 Value
Date                  
2010-01-31  126.177708
2010-02-28  130.301538
2010-03-31  130.460988
2010-04-30  129.393939
2010-05-31  128.309386
2010-06-30  125.838087
2010-07-31  125.272327
2010-08-31  124.898799
2010-09-30  127.339331
2010-10-31  126.904762
2010-11-30  127.243590
2010-12-31  126.904762
2011-01-31  126.603900
2011-02-28  124.720000
2011-03-31  123.219178
2011-04-30  120.984665
2011-05-31  118.934348
2011-06-30  120.123132
2011-07-31  119.937370
2011-08-31  118.646081
2011-09-30  117.481203
2011-10-31  118.529027
2011-11-30  117.320261
2011-12-31  116.387960
2012-01-31  118.652482
2012-02-29  119.257283
2012-03-31  118.123209
2012-04-30  119.173729
2012-05-31  119.637804
2012-06-30  123.526845
...                ...
2015-11-30  179.891293
2015-12-31  178.186358
2016-01-31  180.221820
2016-02-29  183.287469
2016-03-31  184.621791
2016-04-30  192.185983
2016-05-31  189.843015
2016-06-30  189.060416
2016-07-31  177.376171
2016-08-31  190.073624
2016-09-30 

                 Value
Date                  
2010-01-31  231.108871
2010-02-28  233.026961
2010-03-31  239.247813
2010-04-30  244.745570
2010-05-31  244.155844
2010-06-30  235.498976
2010-07-31  230.736474
2010-08-31  234.141093
2010-09-30  217.641467
2010-10-31  209.537572
2010-11-30  205.942241
2010-12-31  209.341500
2011-01-31  210.523841
2011-02-28  209.537572
2011-03-31  213.155370
2011-04-30  213.997821
2011-05-31  222.136752
2011-06-30  227.941176
2011-07-31  225.997340
2011-08-31  222.830087
2011-09-30  217.334608
2011-10-31  217.967938
2011-11-30  217.821782
2011-12-31  214.997170
2012-01-31  219.230769
2012-02-29  233.349105
2012-03-31  229.830190
2012-04-30  238.778514
2012-05-31  238.294314
2012-06-30  223.873047
...                ...
2015-11-30  313.954545
2015-12-31  303.030303
2016-01-31  308.641975
2016-02-29  314.849624
2016-03-31  316.820785
2016-04-30  318.000000
2016-05-31  313.973252
2016-06-30  324.841862
2016-07-31  316.833333
2016-08-31  331.610044
2016-09-30 

                 Value
Date                  
2010-01-31  151.306185
2010-02-28  148.730611
2010-03-31  150.876747
2010-04-30  150.320173
2010-05-31  152.276375
2010-06-30  149.931235
2010-07-31  148.331870
2010-08-31  150.844900
2010-09-30  138.429664
2010-10-31  135.927104
2010-11-30  134.122973
2010-12-31  131.911818
2011-01-31  142.196943
2011-02-28  142.392587
2011-03-31  144.196464
2011-04-30  143.689678
2011-05-31  145.919382
2011-06-30  145.949419
2011-07-31  145.818864
2011-08-31  145.099905
2011-09-30  145.258286
2011-10-31  145.887728
2011-11-30  146.788793
2011-12-31  146.666667
2012-01-31  143.885313
2012-02-29  143.266476
2012-03-31  148.001480
2012-04-30  151.319470
2012-05-31  153.277118
2012-06-30  153.002309
...                ...
2015-11-30  205.696597
2015-12-31  207.827217
2016-01-31  203.488372
2016-02-29  205.217125
2016-03-31  207.720382
2016-04-30  208.333333
2016-05-31  210.560810
2016-06-30  220.036878
2016-07-31  221.714921
2016-08-31  221.164603
2016-09-30 

In [21]:
# Concatenating all the data
#listing_sqf_concat = data_data
listing_sqf_concat = pd.concat(listing_sqf_concat)
# Getting month and year from the column "date"
listing_sqf_concat['Year'] = pd.DatetimeIndex(listing_sqf_concat.index).year
listing_sqf_concat['Month'] = pd.DatetimeIndex(listing_sqf_concat.index).month
rename_listing_sqf_concat = listing_sqf_concat.rename(columns={"Value":"Median Listing Price Per sqf"})
rename_listing_sqf_concat

,Median Listing Price Per sqf,Zip Code,Year,Month
Date,,,,
2010-01-31,342.530800,78701,2010,1
2010-02-28,342.245287,78701,2010,2
2010-03-31,339.803625,78701,2010,3
2010-04-30,330.220815,78701,2010,4
2010-05-31,332.045089,78701,2010,5
2010-06-30,329.467580,78701,2010,6
2010-07-31,327.641105,78701,2010,7
2010-08-31,331.973684,78701,2010,8
2010-09-30,331.474597,78701,2010,9


In [22]:
# Export the "home value" data to a csv file
rename_listing_sqf_concat.to_csv("../data/int/output_quandl/6_Output.median_listing_price_perSQF.csv", index = False ,header = True)

## SALES

In [23]:
# For all the zip codes obtained
x=0
data_zip = []
data_data = []
for z in zips:
    # set quandl api key
    quandl.ApiConfig.api_key = API_KEY
    # set the indicator code to query
    # indicator: Zillow Home Value Index
    indicator_code = "SALES"
    # get quandl code to query
#     quandl_code = get_quandl_code(area_category, zips[x], indicator_code)
    quandl_code = get_quandl_code(area_category, z, indicator_code)
#     print(z)
    # when it finds a zip code where there is not houses for sale (example: 78712 UT)
    try:
        # get housing indicator
        data = quandl.get(quandl_code, start_date="1800-01-01", end_date="2018-04-30")
        pprint(data)
        data_zip.append(z)
        # Appending all the data for every zip code
        data_add_zip = data
        data_add_zip["Zip Code"] = z
        data_data.append(data_add_zip)
    except (KeyboardInterrupt, SystemExit):
        raise
    except Exception as exp:
        print(f"caught exception {exp}")
        continue
    x +=1
    
# Copying the data stored
sales_concat = data_data

            Value
Date             
2008-06-30   88.0
2008-07-31   81.0
2008-08-31   71.0
2008-09-30   40.0
2008-10-31   36.0
2008-11-30    8.0
2008-12-31    6.0
2009-01-31    1.0
2009-02-28    7.0
2009-03-31    9.0
2009-04-30    4.0
2009-05-31    9.0
2009-06-30   16.0
2009-07-31   16.0
2009-08-31   13.0
2009-09-30   17.0
2009-10-31   11.0
2009-11-30   13.0
2009-12-31   10.0
2010-01-31    2.0
2010-02-28    4.0
2010-03-31   12.0
2010-04-30    6.0
2010-05-31   16.0
2010-06-30   21.0
2010-07-31   15.0
2010-08-31   16.0
2010-09-30   17.0
2010-10-31   14.0
2010-11-30    8.0
...           ...
2013-12-31   17.0
2014-01-31    7.0
2014-02-28   10.0
2014-03-31   16.0
2014-04-30   23.0
2014-05-31   28.0
2014-06-30   12.0
2014-07-31   12.0
2014-08-31   14.0
2014-09-30   11.0
2014-10-31   10.0
2014-11-30   15.0
2014-12-31    1.0
2015-01-31    8.0
2015-02-28   11.0
2015-03-31    6.0
2015-04-30   20.0
2015-05-31   11.0
2015-06-30   20.0
2015-07-31   14.0
2015-08-31   15.0
2015-09-30   14.0
2015-10-31

            Value
Date             
2008-06-30   16.0
2008-07-31    9.0
2008-08-31    5.0
2008-09-30    3.0
2008-10-31    7.0
2008-11-30    7.0
2008-12-31    5.0
2009-01-31    2.0
2009-02-28    3.0
2009-03-31    3.0
2009-04-30    6.0
2009-05-31    3.0
2009-06-30    6.0
2009-07-31    9.0
2009-08-31    5.0
2009-09-30    1.0
2009-10-31    5.0
2009-11-30    5.0
2009-12-31    5.0
2010-01-31    2.0
2010-02-28    1.0
2010-03-31    2.0
2010-04-30    8.0
2010-05-31    3.0
2010-06-30    3.0
2010-07-31    4.0
2010-08-31    6.0
2010-09-30    2.0
2010-10-31    3.0
2010-11-30    3.0
...           ...
2013-09-30    2.0
2013-10-31    5.0
2013-11-30    2.0
2013-12-31    2.0
2014-02-28    1.0
2014-04-30    2.0
2014-05-31    3.0
2014-06-30    2.0
2014-07-31    2.0
2014-08-31    3.0
2014-10-31    1.0
2014-11-30    1.0
2014-12-31    3.0
2015-01-31    2.0
2015-02-28    1.0
2015-04-30    3.0
2015-05-31    1.0
2015-06-30    1.0
2015-07-31   10.0
2015-08-31    4.0
2015-09-30    2.0
2015-10-31    6.0
2015-11-30

            Value
Date             
2008-06-30   20.0
2008-07-31   15.0
2008-08-31   21.0
2008-09-30   14.0
2008-10-31   12.0
2008-11-30    7.0
2008-12-31   11.0
2009-01-31   17.0
2009-02-28    6.0
2009-03-31    9.0
2009-04-30   12.0
2009-05-31    8.0
2009-06-30    9.0
2009-07-31   14.0
2009-08-31    5.0
2009-09-30   14.0
2009-10-31   12.0
2009-11-30   12.0
2009-12-31    8.0
2010-01-31    5.0
2010-02-28    6.0
2010-03-31    9.0
2010-04-30    8.0
2010-05-31    5.0
2010-06-30   15.0
2010-07-31    2.0
2010-08-31    9.0
2010-09-30   20.0
2010-10-31    9.0
2010-11-30    3.0
...           ...
2014-01-31    3.0
2014-02-28    4.0
2014-03-31    6.0
2014-04-30    5.0
2014-05-31   11.0
2014-06-30    7.0
2014-07-31    8.0
2014-08-31    5.0
2014-09-30    3.0
2014-10-31    3.0
2014-11-30    3.0
2014-12-31    7.0
2015-01-31    3.0
2015-02-28    3.0
2015-03-31    5.0
2015-04-30    5.0
2015-05-31    7.0
2015-06-30    6.0
2015-07-31   15.0
2015-08-31   12.0
2015-09-30   21.0
2015-10-31   26.0
2015-11-30

            Value
Date             
2008-06-30   40.0
2008-07-31   29.0
2008-08-31   45.0
2008-09-30   21.0
2008-10-31   29.0
2008-11-30   15.0
2008-12-31   38.0
2009-01-31   26.0
2009-02-28   21.0
2009-03-31   15.0
2009-04-30   20.0
2009-05-31   23.0
2009-06-30   39.0
2009-07-31   27.0
2009-08-31   35.0
2009-09-30   21.0
2009-10-31   21.0
2009-11-30   21.0
2009-12-31   48.0
2010-01-31   38.0
2010-02-28   24.0
2010-03-31   49.0
2010-04-30   34.0
2010-05-31   48.0
2010-06-30   87.0
2010-07-31   37.0
2010-08-31   26.0
2010-09-30   25.0
2010-10-31   28.0
2010-11-30   29.0
...           ...
2014-01-31   22.0
2014-02-28   33.0
2014-03-31   41.0
2014-04-30   32.0
2014-05-31   52.0
2014-06-30   59.0
2014-07-31   47.0
2014-08-31   46.0
2014-09-30   36.0
2014-10-31   28.0
2014-11-30   36.0
2014-12-31   46.0
2015-01-31   27.0
2015-02-28   15.0
2015-03-31   21.0
2015-04-30   30.0
2015-05-31   21.0
2015-06-30   23.0
2015-07-31   34.0
2015-08-31   50.0
2015-09-30   43.0
2015-10-31   32.0
2015-11-30

            Value
Date             
2008-06-30  119.0
2008-07-31  105.0
2008-08-31  128.0
2008-09-30  132.0
2008-10-31  104.0
2008-11-30   61.0
2008-12-31   74.0
2009-01-31   42.0
2009-02-28   60.0
2009-03-31   62.0
2009-04-30   58.0
2009-05-31   87.0
2009-06-30  104.0
2009-07-31   80.0
2009-08-31   93.0
2009-09-30   74.0
2009-10-31   87.0
2009-11-30  104.0
2009-12-31   61.0
2010-01-31   33.0
2010-02-28   39.0
2010-03-31   85.0
2010-04-30   93.0
2010-05-31   72.0
2010-06-30   96.0
2010-07-31   49.0
2010-08-31   41.0
2010-09-30   55.0
2010-10-31   36.0
2010-11-30   43.0
...           ...
2014-01-31   13.0
2014-02-28   22.0
2014-03-31   39.0
2014-04-30   33.0
2014-05-31   39.0
2014-06-30   37.0
2014-07-31   35.0
2014-08-31   27.0
2014-09-30   32.0
2014-10-31   25.0
2014-11-30   24.0
2014-12-31   33.0
2015-01-31   28.0
2015-02-28   26.0
2015-03-31   20.0
2015-04-30   39.0
2015-05-31   27.0
2015-06-30   45.0
2015-07-31   78.0
2015-08-31   84.0
2015-09-30   97.0
2015-10-31   54.0
2015-11-30

            Value
Date             
2008-06-30   40.0
2008-07-31   32.0
2008-08-31   34.0
2008-09-30   25.0
2008-10-31   23.0
2008-11-30   10.0
2008-12-31   16.0
2009-01-31    6.0
2009-02-28   13.0
2009-03-31   26.0
2009-04-30   16.0
2009-05-31   35.0
2009-06-30   21.0
2009-07-31   34.0
2009-08-31   32.0
2009-09-30   29.0
2009-10-31   23.0
2009-11-30   43.0
2009-12-31   16.0
2010-01-31   13.0
2010-02-28   10.0
2010-03-31   20.0
2010-04-30   34.0
2010-05-31   24.0
2010-06-30   21.0
2010-07-31   17.0
2010-08-31   24.0
2010-09-30   15.0
2010-10-31   18.0
2010-11-30   17.0
...           ...
2014-01-31   12.0
2014-02-28   12.0
2014-03-31   19.0
2014-04-30   14.0
2014-05-31   28.0
2014-06-30   27.0
2014-07-31   10.0
2014-08-31   21.0
2014-09-30    9.0
2014-10-31    9.0
2014-11-30    9.0
2014-12-31   15.0
2015-01-31    6.0
2015-02-28   16.0
2015-03-31   17.0
2015-04-30   11.0
2015-05-31   10.0
2015-06-30   18.0
2015-07-31   48.0
2015-08-31   25.0
2015-09-30   44.0
2015-10-31   28.0
2015-11-30

In [24]:
# Concatenating all the data
#sales_concat = data_data
sales_concat = pd.concat(sales_concat)
# Getting month and year from the column "date"
sales_concat['Year'] = pd.DatetimeIndex(sales_concat.index).year
sales_concat['Month'] = pd.DatetimeIndex(sales_concat.index).month
rename_sales_concat = sales_concat.rename(columns={"Value":"Sales"})
rename_sales_concat

,Sales,Zip Code,Year,Month
Date,,,,
2008-06-30,88.0,78701,2008,6
2008-07-31,81.0,78701,2008,7
2008-08-31,71.0,78701,2008,8
2008-09-30,40.0,78701,2008,9
2008-10-31,36.0,78701,2008,10
2008-11-30,8.0,78701,2008,11
2008-12-31,6.0,78701,2008,12
2009-01-31,1.0,78701,2009,1
2009-02-28,7.0,78701,2009,2


In [25]:
# Export the "home value" data to a csv file
rename_sales_concat.to_csv("../data/int/output_quandl/7_Output.sales.csv", index = False ,header = True)

## Median Rental Price - All Homes|MRPAH

In [26]:
# For all the zip codes obtained
x=0
data_zip = []
data_data = []
for z in zips:
    # set quandl api key
    quandl.ApiConfig.api_key = API_KEY
    # set the indicator code to query
    # indicator: Zillow Home Value Index
    indicator_code = "MRPAH"
    # get quandl code to query
#     quandl_code = get_quandl_code(area_category, zips[x], indicator_code)
    quandl_code = get_quandl_code(area_category, z, indicator_code)
#     print(z)
    # when it finds a zip code where there is not houses for sale (example: 78712 UT)
    try:
        # get housing indicator
        data = quandl.get(quandl_code, start_date="1800-01-01", end_date="2018-04-30")
        pprint(data)
        data_zip.append(z)
        # Appending all the data for every zip code
        data_add_zip = data
        data_add_zip["Zip Code"] = z
        data_data.append(data_add_zip)
    except (KeyboardInterrupt, SystemExit):
        raise
    except Exception as exp:
        print(f"caught exception {exp}")
        continue
    x +=1
    
rental_concat = data_data

             Value
Date              
2011-01-31  1850.0
2011-02-28  1925.0
2011-03-31  2000.0
2011-04-30  2200.0
2011-05-31  2020.0
2011-06-30  2100.0
2011-07-31  2075.0
2011-08-31  2100.0
2011-09-30  2050.0
2011-10-31  2000.0
2011-11-30  2016.0
2011-12-31  1995.0
2012-01-31  2050.0
2012-02-29  2095.0
2012-03-31  2200.0
2012-04-30  2200.0
2012-05-31  2100.0
2012-06-30  2100.0
2012-07-31  2100.0
2012-08-31  2000.0
2012-09-30  2000.0
2012-10-31  2100.0
2012-11-30  2200.0
2012-12-31  2300.0
2013-01-31  2100.0
2013-02-28  2100.0
2013-03-31  2100.0
2013-04-30  2108.0
2013-05-31  2140.0
2013-06-30  2260.0
...            ...
2015-03-31  2496.0
2015-04-30  2495.0
2015-05-31  2472.5
2015-06-30  2345.0
2015-07-31  2200.0
2015-08-31  2300.0
2015-09-30  2300.0
2015-10-31  2395.0
2015-11-30  2372.5
2015-12-31  2495.0
2016-01-31  2472.0
2016-02-29  2400.0
2016-03-31  2375.0
2016-04-30  2473.0
2016-05-31  2498.5
2016-06-30  2450.0
2016-07-31  2250.0
2016-08-31  2449.0
2016-09-30  2543.0
2016-10-31  

             Value
Date              
2012-12-31  1010.0
2013-01-31   860.0
2013-02-28   895.0
2013-03-31   880.0
2013-04-30   885.0
2013-05-31   925.0
2013-06-30   908.0
2013-07-31   954.0
2013-08-31   995.0
2013-09-30   955.0
2013-10-31   954.0
2013-11-30   950.0
2013-12-31   928.0
2014-01-31   950.0
2014-02-28   967.5
2014-03-31  1055.0
2014-04-30  1080.0
2014-05-31   987.0
2014-06-30   979.0
2014-07-31  1012.5
2014-08-31  1002.5
2014-09-30   997.5
2014-10-31   987.0
2014-11-30   956.0
2014-12-31   967.5
2015-01-31   940.5
2015-02-28   940.5
2015-03-31   950.0
2015-04-30   967.5
2015-05-31  1035.0
2015-06-30  1047.5
2015-07-31  1047.5
2015-08-31  1025.0
2015-09-30  1075.0
2015-10-31  1035.0
2015-11-30  1000.0
2015-12-31   954.0
2016-01-31   995.0
2016-02-29  1000.0
2016-03-31  1005.0
2016-04-30   967.5
2016-05-31   954.0
2016-06-30   967.5
2016-07-31  1002.5
2016-08-31   967.5
2016-09-30   967.5
2016-10-31   967.5
2016-11-30   952.0
2016-12-31   955.0
2017-01-31  1005.0
2017-02-28  

             Value
Date              
2013-02-28   950.0
2013-03-31  1000.0
2013-04-30  1075.0
2013-05-31  1090.0
2013-06-30  1102.0
2013-07-31  1028.0
2013-08-31  1100.0
2013-09-30  1194.0
2013-10-31  1159.5
2013-11-30  1250.0
2013-12-31  1184.0
2014-01-31  1150.0
2014-02-28  1195.0
2014-03-31  1265.0
2014-04-30  1300.0
2014-05-31  1188.5
2014-06-30  1152.5
2014-07-31  1197.5
2014-08-31  1193.0
2014-09-30  1188.5
2014-10-31  1152.5
2014-11-30  1150.0
2014-12-31  1090.0
2015-01-31  1090.0
2015-02-28  1135.0
2015-03-31  1090.0
2015-04-30  1150.0
2015-05-31  1222.5
2015-06-30  1240.0
2015-07-31  1295.0
...            ...
2015-11-30  1336.0
2015-12-31  1250.0
2016-01-31  1240.0
2016-02-29  1245.0
2016-03-31  1250.0
2016-04-30  1188.5
2016-05-31  1195.0
2016-06-30  1250.0
2016-07-31  1260.0
2016-08-31  1250.0
2016-09-30  1195.0
2016-10-31  1222.5
2016-11-30  1184.0
2016-12-31  1195.0
2017-01-31  1347.0
2017-02-28  1400.0
2017-03-31  1475.0
2017-04-30  1512.5
2017-05-31  1495.0
2017-06-30  

             Value
Date              
2013-02-28  1020.0
2013-03-31   945.0
2013-04-30   957.5
2013-05-31   925.0
2013-06-30   997.0
2013-07-31   956.5
2013-08-31   987.5
2013-09-30  1006.0
2013-10-31  1134.0
2013-11-30  1118.0
2013-12-31  1038.5
2014-01-31  1028.5
2014-02-28  1020.0
2014-03-31  1022.5
2014-04-30  1079.0
2014-05-31  1054.0
2014-06-30  1054.0
2014-07-31  1225.0
2014-08-31  1232.0
2014-09-30  1064.5
2014-10-31  1064.5
2014-11-30  1045.0
2014-12-31  1038.5
2015-01-31  1032.0
2015-02-28  1045.0
2015-03-31  1064.5
2015-04-30  1079.0
2015-05-31  1085.0
2015-06-30  1232.0
2015-07-31  1224.5
2015-08-31  1227.5
2015-09-30  1250.0
2015-10-31  1212.5
2015-11-30  1150.0
2015-12-31  1092.5
2016-01-31  1150.0
2016-02-29  1183.0
2016-03-31  1150.0
2016-04-30  1020.0
2016-05-31  1017.5
2016-06-30  1064.5
2016-07-31  1134.0
2016-08-31  1118.0
2016-09-30  1064.5
2016-10-31  1085.0
2016-11-30  1050.0
2016-12-31  1050.0
2017-01-31  1166.5
2017-02-28  1230.0
2017-03-31  1307.0
2017-04-30  

In [27]:
# Concatenating all the data
rental_concat = data_data
rental_concat = pd.concat(rental_concat)
# Getting month and year from the column "date"
rental_concat['Year'] = pd.DatetimeIndex(rental_concat.index).year
rental_concat['Month'] = pd.DatetimeIndex(rental_concat.index).month
rename_rental_concat = rental_concat.rename(columns={"Value":"Median Rental Price"})
rename_rental_concat

,Median Rental Price,Zip Code,Year,Month
Date,,,,
2011-01-31,1850.0,78701,2011,1
2011-02-28,1925.0,78701,2011,2
2011-03-31,2000.0,78701,2011,3
2011-04-30,2200.0,78701,2011,4
2011-05-31,2020.0,78701,2011,5
2011-06-30,2100.0,78701,2011,6
2011-07-31,2075.0,78701,2011,7
2011-08-31,2100.0,78701,2011,8
2011-09-30,2050.0,78701,2011,9


In [28]:
# Export the "home value" data to a csv file
rename_rental_concat.to_csv("../data/int/output_quandl/8_Output.rental_prices.csv", index = False ,header = True)

## Merging all the tables

In [29]:
quandl_merge = pd.merge(rename_home_value_concat,rename_inventory_concat, how="left")
quandl_merge = pd.merge(quandl_merge,rename_price_cut_concat, how="left")
quandl_merge = pd.merge(quandl_merge,rename_price_reduction_concat, how="left")
quandl_merge = pd.merge(quandl_merge,rename_listing_concat, how="left")
quandl_merge = pd.merge(quandl_merge,rename_listing_sqf_concat, how="left")
quandl_merge = pd.merge(quandl_merge,rename_sales_concat, how="left")
quandl_merge = pd.merge(quandl_merge,rename_rental_concat, how="left")
quandl_merge

,Home Value Index,Zip Code,Year,Month,Inventory Measure,Median Price Cut,Median Price of Reduction,Median Listing Price,Median Listing Price Per sqf,Sales,Median Rental Price
0,152500.0,78701,1996,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,153600.0,78701,1996,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,154600.0,78701,1996,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,155500.0,78701,1996,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,156300.0,78701,1996,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,157300.0,78701,1996,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,158300.0,78701,1996,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,159300.0,78701,1996,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,160400.0,78701,1996,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,161900.0,78701,1997,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
quandl_merge.to_csv("../data/int/1_Output_quandl.csv", index=False, header=True)